In [ ]:
import pandas as pd
import country_converter as coco


from dfpp.transformation.column_name_template import (
    CANONICAL_COLUMN_NAMES,
    DIMENSION_COLUMN_PREFIX,
    SERIES_PROPERTY_PREFIX,
    sort_columns_canonically,
    ensure_canonical_columns,
)
from dfpp.publishing import publish_series

cc = coco.CountryConverter()

In [ ]:
URL = "https://energydata.info/dataset/b33e5af4-bd51-4ee0-a062-29438471db27/resource/6938ec3a-f7bb-4493-86ba-f28faa62f139/download/eleccap_20220404-201215.xlsx"
df_source = pd.read_excel(URL, header=1)

In [30]:
df = df_source.copy()

In [31]:
df.columns = ["country", DIMENSION_COLUMN_PREFIX + "energy_technology", DIMENSION_COLUMN_PREFIX + "grid_connection", "year", "value"]
df.ffill(inplace=True)

df["value"] = df["value"].replace({"..": None})
df["alpha_3_code"] = cc.pandas_convert(df["country"], to="ISO3")
df = df[df["alpha_3_code"] != "not found"].reset_index(drop=True)

df["year"] = df["year"].astype(int)

df[SERIES_PROPERTY_PREFIX + "unit"] = "Megawatt"
df["value"] = df["value"].astype("float")
series_id = "irena_eleccap"

df["source"] = "https://energydata.info/"
df["series_id"] = "irena_eleccap"
df["series_name"] = """Installed electricity capacity by country/area (MW) by Country/area, Technology, Grid connection and Year"""

to_select_columns = [
        col
        for col in df.columns
        if any([col.startswith(DIMENSION_COLUMN_PREFIX),
                col.startswith(SERIES_PROPERTY_PREFIX)]) and col not in CANONICAL_COLUMN_NAMES
    ]
df = ensure_canonical_columns(df)
df = df[CANONICAL_COLUMN_NAMES + to_select_columns]
df = sort_columns_canonically(df)

2024-11-16 15:54:10,456 - WARNING - country_converter - The renewable power capacity data shown in these tables represents the maximum net generating capacity of power plants and other installations that use renewable energy sources to produce electricity. For most countries and technologies, the data reflects the capacity installed and connected at the end of the calendar year. not found in regex
2024-11-16 15:54:10,457 - WARNING - country_converter - Source: not found in regex
2024-11-16 15:54:10,464 - WARNING - country_converter - IRENA (2021), Renewable capacity statistics 2021 International Renewable Energy Agency (IRENA), Abu Dhabi not found in regex
2024-11-16 15:54:10,469 - WARNING - country_converter - Units: not found in regex
2024-11-16 15:54:10,481 - WARNING - country_converter - Megawatt MW not found in regex
2024-11-16 15:54:10,767 - WARNING - country_converter - Database: not found in regex
2024-11-16 15:54:10,778 - WARNING - country_converter - IRENA Renewable Energy St

In [32]:
await publish_series(series_id = series_id, df_series = df, source_folder = "energydata_info")

2024-11-16 15:54:11,593 - INFO - _universal - Request URL: 'https://sdgigeneralstorage.blob.core.windows.net/dfx-etl-pipeline-dev/test/energydata_info/parquet/irena_eleccap.parquet?sv=REDACTED&st=REDACTED&se=REDACTED&sr=REDACTED&sp=REDACTED&sig=REDACTED'
Request method: 'PUT'
Request headers:
    'Content-Length': '74133'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/xml'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '420f6bea-a422-11ef-adc9-acbc327b2b3d'
    'User-Agent': 'azsdk-python-storage-blob/12.21.0 Python/3.11.10 (macOS-12.7.5-x86_64-i386-64bit)'
A body is sent with the request
2024-11-16 15:54:12,281 - INFO - _universal - Response status: 201
Response headers:
    'Content-Length': '0'
    'Content-MD5': 'REDACTED'
    'Last-Modified': 'Sat, 16 Nov 2024 13:54:12 GMT'
    'Etag': '"0x8DD064626B25066"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-req